__Labeling Tweets__

To build a target variable column, i,e., to label the sentiment of the given text, we will compare several sentiment analyzer tools which are widely available for classifying the data.Through the remaining sections, we’ll compare and discuss classification results using several well-known NLP libraries in Python. The methods described below fall under five broad categories as below:
VADER, Textblob, SentiWordNet lexicon from NTLK, StanfordCoreNLP, Afinn
The Positive, Negative and Neutral scores represent the proportion of text that falls in these categories which have been normalized between -1(most negative) and +1 (most positive). 

In [1]:
# import the necessary modules
import numpy as np
import pandas as pd

In [2]:
# import the csv file as Pandas dataframe
df = pd.read_csv("@tweets14.csv")

In [3]:
# shape of DataFrame
df.shape

(18367, 15)

In [5]:
# Drop All rows with missing values
df = df.dropna()
df.shape

(18366, 15)

In [6]:
# Convert the created_at column to np.datetime object
df['Date of Tweet'] = pd.to_datetime(df['Date of Tweet'])

# Print created_at to see new format
print(df['Date of Tweet'].head())

# Set the index of ds_tweets to created_at
df.set_index('Date of Tweet', inplace = True)

0   2013-08-09 18:06:33
1   2010-09-02 06:23:41
2   2015-10-07 01:32:48
3   2011-04-28 01:56:15
4   2019-11-07 19:03:58
Name: Date of Tweet, dtype: datetime64[ns]


In [8]:
df['clean_text'] = df['clean_text'].astype(str)
df['clean_text'].iloc[2]

'If anything the circus about the impeachment comes late that guy and his fascist minions have destroyed the USA thx to the ABBERANTLY RIDICULOUSLY STUPIDLY REDUNDANT OBSOLETE ELECTORAL COLLEGE and APATHY of millions of eligible voters that didnt vote in'

__1. SentiWordNet lexicon:__

First let's label the tweets as either positive, negative or neutral using SentiWordNet lexicon. Words are associated with a sentiment score included between -1 and 1. Words are in the form lemma#PoS and are aligned with WordNet lists that include adjectives, nouns, verbs and adverbs.

In [9]:
from nltk import pos_tag, map_tag
import time
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.tag import pos_tag,map_tag
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize 

pstem = PorterStemmer()
lem = WordNetLemmatizer()

df_copy = df.copy()
df_copy = df_copy.reset_index()
def pos_senti(df_copy):#takes
    li_swn=[]
    li_swn_pos=[]
    li_swn_neg=[]
    missing_words=[]
    for i in range(len(df_copy.index)):
        text = df_copy.loc[i]['clean_text']
        tokens = word_tokenize(str(text))
        tagged_sent = pos_tag(tokens)
        store_it = [(word, map_tag('en-ptb', 'universal', tag)) for word, tag in tagged_sent]
        #print("Tagged Parts of Speech:",store_it)

        pos_total=0
        neg_total=0
        for word,tag in store_it:
            if(tag=='NOUN'):
                tag='n'
            elif(tag=='VERB'):
                tag='v'
            elif(tag=='ADJ'):
                tag='a'
            elif(tag=='ADV'):
                tag = 'r'
            else:
                tag='nothing'

            if(tag!='nothing'):
                concat = word+'.'+tag+'.01'
                try:
                    this_word_pos=swn.senti_synset(concat).pos_score()
                    this_word_neg=swn.senti_synset(concat).neg_score()
                    #print(word,tag,':',this_word_pos,this_word_neg)
                except Exception as e:
                    wor = lem.lemmatize(word)
                    concat = wor+'.'+tag+'.01'
                    # Checking if there's a possiblity of lemmatized word be accepted into SWN corpus
                    try:
                        this_word_pos=swn.senti_synset(concat).pos_score()
                        this_word_neg=swn.senti_synset(concat).neg_score()
                    except Exception as e:
                        wor = pstem.stem(word)
                        concat = wor+'.'+tag+'.01'
                        # Checking if there's a possiblity of lemmatized word be accepted
                        try:
                            this_word_pos=swn.senti_synset(concat).pos_score()
                            this_word_neg=swn.senti_synset(concat).neg_score()
                        except:
                            missing_words.append(word)
                            continue
                pos_total+=this_word_pos
                neg_total+=this_word_neg
        li_swn_pos.append(pos_total)
        li_swn_neg.append(neg_total)

        if(pos_total!=0 or neg_total!=0):
            if(pos_total>neg_total):
                li_swn.append(1)
            else:
                li_swn.append(-1)
        else:
            li_swn.append(0)
    df_copy.insert(2,"pos_score",li_swn_pos,True)
    df_copy.insert(3,"neg_score",li_swn_neg,True)
    df_copy.insert(4,"sent_score",li_swn,True)
    return df_copy
    # end-of pos-tagging&sentiment
df3 = pos_senti(df_copy)

In [10]:
#counts of unique positive, negative and neutral values
df3.sent_score.value_counts()

 1    9000
-1    6910
 0    2456
Name: sent_score, dtype: int64

__2. AFINN:__

AFINN is a manually labeled by Finn Årup Nielsen in 2009–2011 list of English words rated for valence with an integer between minus five (negative) and plus five (positive) [5]

In [11]:
# Afinn sentiment LABELING
from afinn import Afinn
af = Afinn()
count_total=0
count_pos=0
count_neut=0

count_neg=0
li_af = []
for i in range(len(df_copy.index)):
    sent = str(df_copy.loc[i]['clean_text'])
    if(af.score(sent)>0):
        count_pos=count_pos+1
        count_total=count_total+1
        li_af.append(1)
    elif(af.score(sent)<0):
        count_neg=count_neg+1
        count_total=count_total+1
        li_af.append(-1)
    else:
        li_af.append(0)
        count_total=count_total+1
        count_neut+=1




print("Total tweets:",len(df_copy.index))
print("Total tweets with sentiment:",count_total)
print("positive tweets:",count_pos)
print("negative tweets:",count_neg)
print("neutral tweets:",count_neut)

Total tweets: 18366
Total tweets with sentiment: 18366
positive tweets: 7387
negative tweets: 4514
neutral tweets: 6465


__3. TextBlob:__

TextBlob is a popular Python library for processing textual data. It is built on top of NLTK, another popular Natural Language Processing toolbox for Python. TextBlob uses a sentiment lexicon (consisting of predefined words) to assign scores for each word, which are then averaged out using a weighted average to give an overall sentence sentiment score. Three scores: “polarity”, “subjectivity” and “intensity” are calculated for each word.

In [12]:
#TextBlob SENTIMENT LABELING
from textblob import TextBlob
count_total=0
count_pos=0
count_neg=0
count_neut=0

li_tb = []
for i in range(len(df_copy.index)):
    sent = TextBlob(str(df_copy.loc[i]["clean_text"]))
    if(sent.sentiment.polarity>0):
        count_pos=count_pos+1
        count_total=count_total+1
        li_tb.append(1)
    elif(sent.sentiment.polarity<0):
        count_neg=count_neg+1
        count_total=count_total+1
        li_tb.append(-1)
    else:
        li_tb.append(0)
        count_neut+=1

        count_total=count_total+1


print("Total tweets:",len(df_copy.index))
print("Total tweets with sentiment:",count_total)
print("positive tweets:",count_pos)
print("negative tweets:",count_neg)
print("neutral tweets:",count_neut)

Total tweets: 18366
Total tweets with sentiment: 18366
positive tweets: 7682
negative tweets: 3067
neutral tweets: 7617


__4. VADER:__

VADER (Valence Aware Dictionary and sentiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.Once VADER is installed SentimentIntensityAnalyser object will be called to classify texts as below:

In [13]:
# Load SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Convert the created_at column to np.datetime object
df6 = df.copy()

# Instantiate new SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# Generate sentiment scores
sentiment_scores = df6['clean_text'].apply(sid.polarity_scores)

In [14]:
df6["score"] = sentiment_scores.apply(lambda x: x['compound'])

In [15]:
# Load SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

count_total=0
count_pos=0
count_neg=0
count_neut=0


for i in df6["score"]:
    if i >0:
        count_pos=count_pos+1
    elif i <0:
        count_neg = count_neg +1
    else:
        count_neut = count_neut +1
        
 

print("positive tweets:",count_pos)
print("negative tweets:",count_neg)
print("neutral tweets:",count_neut)
conditions = [
    (df6['score'] >0),
    (df6['score'] <0),
    (df6['score'] == 0)]
choices = [1,-1,0]
df6['sentiment'] = np.select(conditions, choices )


positive tweets: 8072
negative tweets: 4995
neutral tweets: 5299


In [16]:
# print few rows

df6.head()

,User_Id,Total Tweets,Favourites_Count,Followers,User_Verified,User Location,Tweet Id,Tweet Text,Tweet Source,Tweet Retweet,full_text,year,hashtags,clean_text,score,sentiment
Date of Tweet,,,,,,,,,,,,,,,,
2013-08-09 18:06:33,'1658338609','313','657','80','False','','1194457803089276928','RT @MrDuganDVOLP: Worked with Lila and Nate t...,Twitter for Android,'2',@MrDuganDVOLP: Worked with Lila and Nate today...,2013,[],Worked with Lila and Nate today at BGHS Online...,0.6249,1
2010-09-02 06:23:41,'185949950','10396','11498','271','False','','1194457753256775686',"'@verrierf @TMZ We did vote, if you recall Hil...",Twitter for iPhone,'0',"@verrierf @TMZ We did vote, if you recall Hill...",2010,[],We did vote if you recall Hillary won the popu...,0.7579,1
2015-10-07 01:32:48,'3809154554','10916','11841','2101','False',"'Burbank, CA'",'1194457695308222464','@MaddowBlog @MSNBC If anything the circus abo...,Twitter for Android,'0',@MaddowBlog @MSNBC If anything the circus abou...,2015,[],If anything the circus about the impeachment c...,-0.9519,-1
2011-04-28 01:56:15,'289118612','2200777','19420','65108','False',"'Atlanta, GA'",'1194457679592341507','The Democratic nominee against Trump could be...,Media Studio,'0',The Democratic nominee against Trump could bea...,2011,[],The Democratic nominee against Trump could bea...,-0.4019,-1
2019-11-07 19:03:58,'1192518029684461570','9','0','1','False','United States','1194457622914707457','We\xe2\x80\x99re still skewed by population b...,Twitter for iPhone,'0',Were still skewed by population but not quite ...,2019,"['#ElectionResults2019', '#ElectionResults2019...",Were still skewed by population but not quite ...,0.0000,0


__5. StanfordCoreNLP:__
    
StanfordCoreNLP builds on grammatical structures.

In [19]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

df8 = df.copy()
def get_sentiment(text):
    res = nlp.annotate(text,
                       properties={'annotators': 'sentiment,tokenize,ssplit',
                                   'outputFormat': 'json',
                                   'timeout': 1000,
                       })
    return res['sentences'][0]['sentiment']



In [20]:
text_amb = "We did vote if you recall Hillary won the popular vote by over million votes"
get_sentiment(text_amb)

'Negative'

In [21]:
df8["sentiment"] =df8['clean_text'].map(get_sentiment)

In [22]:
pd.set_option('display.max_colwidth', -1)
df8[['year', 'clean_text','sentiment']][1:5]

,year,clean_text,sentiment
Date of Tweet,,,
2010-09-02 06:23:41,2010,We did vote if you recall Hillary won the popular vote by over million votes,Negative
2015-10-07 01:32:48,2015,If anything the circus about the impeachment comes late that guy and his fascist minions have destroyed the USA thx to the ABBERANTLY RIDICULOUSLY STUPIDLY REDUNDANT OBSOLETE ELECTORAL COLLEGE and APATHY of millions of eligible voters that didnt vote in,Negative
2011-04-28 01:56:15,2011,The Democratic nominee against Trump could beat him by far more votes than Hillary Clinton did in and still lose,Negative
2019-11-07 19:03:58,2019,Were still skewed by population but not quite near as much TGI Electoral College #ElectionResults #ElectionResults #ElectoralCollege #Election #YangGang #FeelTheBern #TrumpTrain,Neutral


In [23]:
df8.sentiment.value_counts( )

Negative        9985
Neutral         6591
Positive        1756
Verynegative    28  
Verypositive    6   
Name: sentiment, dtype: int64

In [24]:
df8["clean_text"].iloc[1]

'We did vote if you recall Hillary won the popular vote by over million votes'

In [25]:
df.dtypes

User_Id             object
Total Tweets        object
Favourites_Count    object
Followers           object
User_Verified       object
User Location       object
Tweet Id            object
Tweet Text          object
Tweet Source        object
Tweet Retweet       object
full_text           object
year                int64 
hashtags            object
clean_text          object
dtype: object

In [26]:
conditions = [
    (df8['sentiment'] == 'Negative'),
    (df8['sentiment'] == 'Verynegative'),
    (df8['sentiment']== "Positive"),
    (df8['sentiment'] == "Verypositive" ),
    (df8['sentiment']== "Neutral")
]

choices = [-1,-1,1,1,0]
df8['sentiment_val'] = np.select(conditions, choices )


Now, let's compare all the sentiment analyzer tools:

In [39]:
print("StanfordCoreNLP:")
print(df8[["clean_text","sentiment_val" ,"sentiment"]].iloc[1])

StanfordCoreNLP:
clean_text       We did vote if you recall Hillary won the popular vote by over million votes
sentiment_val    -1                                                                          
sentiment        Negative                                                                    
Name: 2010-09-02 06:23:41, dtype: object


In [41]:
print("Vader:")
print(df6[["clean_text","sentiment"]].iloc[1])

Vader:
clean_text    We did vote if you recall Hillary won the popular vote by over million votes
sentiment     1                                                                           
Name: 2010-09-02 06:23:41, dtype: object


In [43]:
print("SentiWordNet lexicon:")
print(df6[["clean_text","score" ,"sentiment"]].iloc[1])

SentiWordNet lexicon:
clean_text    We did vote if you recall Hillary won the popular vote by over million votes
score         0.7579                                                                      
sentiment     1                                                                           
Name: 2010-09-02 06:23:41, dtype: object


In [32]:
print("TextBlob:")
print(df_copy[["clean_text","sentiment_val" ,"sentiment"]].iloc[1])

TextBlob:
Date of Tweet       2010-09-02 06:23:41                                                                                                       
User_Id             '185949950'                                                                                                               
pos_score           0.375                                                                                                                     
neg_score           0                                                                                                                         
sent_score          1                                                                                                                         
Total Tweets        '10396'                                                                                                                   
Favourites_Count    '11498'                                                                                                         

In [33]:
print("Afinn:")
print(df3.iloc[1])

Afinn:
Date of Tweet       2010-09-02 06:23:41                                                                                                       
User_Id             '185949950'                                                                                                               
pos_score           0.375                                                                                                                     
neg_score           0                                                                                                                         
sent_score          1                                                                                                                         
Total Tweets        '10396'                                                                                                                   
Favourites_Count    '11498'                                                                                                            

In [35]:
#df8.to_csv("@tweets_final1.csv")

__Conclusion:__

As we can see, all the other analyzers classified tweet "We did vote if you recall Hillary won the popular vote by over million votes"  as positive whereas StanfordCoreNLP classified it as negative which is right with respect to "electoral college".  StanfordCoreNLP will be used to label our tweets dataset since it is designed to help evaluate a model’s ability to understand representations of sentence structure, rather than just looking at individual words in isolation.